In [1]:
%load_ext lab_black

Step: Load data

In [2]:
import pandas as pd
import os
import numpy as np
import nltk
from pyprojroot import here

from skimpy import clean_columns
from data_cleaning.fun_hot_encode_limit import fun_hot_encode_limit

path_data = here("./data")
os.chdir(path_data)
data_ra = pd.read_csv("ra_data.csv")
data_ra = clean_columns(data_ra)

nltk.download("vader_lexicon")
data_ra["date"] = pd.to_datetime(data_ra["date"])

11 column names have been cleaned

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /home/azureuser/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
/tmp/ipykernel_6498/814015519.py:16: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  data_ra["date"] = pd.to_datetime(data_ra["date"])


In [6]:
import requests
from bs4 import BeautifulSoup
import pandas as pd


def get_reviews(url):
    headers = {"User-Agent": "Mozilla/5.0"}
    response = requests.get(url, headers=headers)

    soup = BeautifulSoup(response.text, "html.parser")
    reviews = soup.find_all("div", class_="description")

    reviews_list = []

    for review in reviews:
        review_dict = {}
        visible_text = review.find("span", class_="showSec")
        hidden_text = review.find("span", class_="hiddenSec")

        # If there's visible text, add it to the review dictionary
        if visible_text:
            review_dict["text"] = visible_text.text.strip()
        # If there's hidden text, append it to the review dictionary
        if hidden_text:
            review_dict["text"] += " " + hidden_text.text.strip()

        reviews_list.append(review_dict)

    return pd.DataFrame(reviews_list)


url = "https://reviews.webmd.com/drugs/drugreview-64713-humira-subcutaneous"
df = get_reviews(url)
print(df)

                                                 text
0                                                 NaN
1   My dear old mother took Humira for her rheumat...
2   I was at wit's end with approximately 90% or m...
3   This drug is poison. I was on it for 2 years a...
4                                                 NaN
5                                                 NaN
6                                                 NaN
7                                                 NaN
8                                                 NaN
9                                                 NaN
10  My husband had a problem in his nails and been...
11                                                NaN
12                                                NaN
13                                                NaN
14  My grandmother died from this medication. She ...
15                                                NaN
16  I was on methotrexate and it quit working for ...
17                          

In [10]:
import requests
from bs4 import BeautifulSoup
import pandas as pd


def get_reviews(base_url):
    headers = {"User-Agent": "Mozilla/5.0"}

    reviews_list = []

    for page in range(1, 11):  # adjust range as necessary
        url = f"{base_url}?conditionid=&sortval=1&page={page}&next_page=true"
        response = requests.get(url, headers=headers)

        soup = BeautifulSoup(response.text, "html.parser")
        reviews = soup.find_all("div", class_="description")

        for review in reviews:
            review_dict = {}
            visible_text = review.find("span", class_="showSec")
            hidden_text = review.find("span", class_="hiddenSec")

            # If there's visible text, add it to the review dictionary
            if visible_text:
                review_dict["text"] = visible_text.text.strip()
            # If there's hidden text, append it to the review dictionary
            if hidden_text:
                review_dict["text"] += " " + hidden_text.text.strip()

            reviews_list.append(review_dict)

    return pd.DataFrame(reviews_list)


base_url = "https://reviews.webmd.com/drugs/drugreview-64713-humira-subcutaneous"
df = get_reviews(base_url)
print(df)

                                                  text
0                                                  NaN
1    My dear old mother took Humira for her rheumat...
2    I was at wit's end with approximately 90% or m...
3    This drug is poison. I was on it for 2 years a...
4                                                  NaN
..                                                 ...
183  Since January 2009 I am taking Humira without ...
184  Fantastic.  I hope they don't take it away.  F...
185                                                NaN
186                                                NaN
187                                                NaN

[188 rows x 1 columns]


In [11]:
pd.DataFrame(df).dropna().reset_index(drop=True)

,text
0,My dear old mother took Humira for her rheumat...
1,I was at wit's end with approximately 90% or m...
2,This drug is poison. I was on it for 2 years a...
3,My husband had a problem in his nails and been...
4,My grandmother died from this medication. She ...
5,I was on methotrexate and it quit working for ...
6,"I have severe psoriasis, 85% of my body is cov..."
7,I had results in 2 weeks after starting Humira...
8,The pen for Enbrel seems to be better made as ...
9,I have been taking Humira fortnightly with a p...
